# BME280: Bring up the sensor via an USB-to-I2C-Adapter

Date: 2021-05-29

*Wichtig:*  
Der BM**P**280 ist nur ein Drucksensor; der BM**E**280 ist das technische Upgrade und misst zusätzlich Temperatur und Luftfeuchtigkeit!

Datenblatt: https://raw.githubusercontent.com/rm-hull/bme280/master/doc/tech-spec/BME280.pdf  
Python-Lib für Raspi: https://pypi.org/project/RPi.bme280/

## Installation benötigter Python-Bibliotheken

Installation erfolgt in Python-Environment:
```
$ source ~/jupyter-env/bin/activate
$ pip install smbus2 RPi.bme280
```
## Installation des Treibers für den USB-I2C-Adapter

<figure>
  <img src="./images/i2c-to-usb_ch341.jpg" width="400px" height="600" alt="USB-I2C-Adapter" />
  <figcaption>USB-I2C-Adapter auf Basis des CH341-Chips</figcaption>
</figure>

Bei Verwendung eines USB-I2C-Adapters auf Basis des CH341-Chips muss das passende Kernelmodul compiliert und geladen werden.
Es gibt 2 verschiedene Implementierungen, wobei davon nur eine funktioniert:

- https://github.com/allanbian1017/i2c-ch341-usb <span style="color:green">[funktioniert]</span>
- https://github.com/gschorcht/i2c-ch341-usb <span style="color:red">[funktioniert NICHT]</span>

### Prerequisites

To compile the driver, you must have installed current kernel header files.

Even though it is not mandatory, it is highly recommended to use DKMS (dynamic kernel module support) for the installation of the driver. DKMS allows to manage kernel modules whose sources reside outside the kernel source tree. Such modules are then automatically rebuilt when a new kernel version is installed.

To use DKMS, it has to be installed before, e.g., with following command on Debian based systems. The current kernel header files will be installed automatically.

```
# apt update
# apt install dkms
```

### Compile the kernel module and load it

```
$ mkdir ~/drivers && cd ~/drivers
$ git clone https://github.com/allanbian1017/i2c-ch341-usb.git
$ cd i2c-ch341-usb
$ make

$ sudo insmod i2c-ch341-usb.ko
```

### Nacharbeiten

Welche Kernel-Module sind geladen?

```
$ lsmod | grep -i i2c
$ lsmod | grep -i ch341
```

Standardmäßig wird das Kernel-Modul ```ch341``` geladen und vom Modul ```usbserial``` verwendet. Dieses nutzt lediglich die UART-Funktion des CH341-Chips und erzeugt so einen Konflikt mit I2C-Funktion. Erkennbar ist dies mit folgendem Befehl:

```
$dmesg

...
[  225.466832] ch341 1-1.2:1.0: ch341-uart converter detected
[  225.469053] usb 1-1.2: ch341-uart converter now attached to ttyUSB0
...
```

Daher wird dieses Modul zunächst entladen und danach das I2C-Modul ```i2c-ch341-usb``` geladen - falls noch nicht automatisch geschehen.

Entladen und laden der Module:

```
$ sudo rmmod ch341
$ sudo modprobe i2c-ch341-usb
```

Wenn dies ohne Fehlermeldung geklappt hat sollte ```dmesg``` folgendes ausgeben:

```
...
[  551.709710] i2c_ch341_usb: loading out-of-tree module taints kernel.
[  551.710558] i2c i2c-11: connected i2c-ch341-usb device
[  551.710676] usbcore: registered new interface driver i2c-ch341-usb
...
```

### Funktionskontrolle

Nun sollte neben dem Raspberry Pi-eigenen I2C-Bus (```/dev/i2c-1```) eine weitere Bus-Gerätedatei verfügbar sein (z. B. ```/dev/i2c-11```). Angeschlossene I2C-Devices können auf dem Bus erkannt und im Weiteren verwendet werden (hier z. B. der Sensor BME280 mit der Adresse 0x76):

```
$ i2cdetect -y 11

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: -- -- -- -- -- -- 76 --
```

### Automatisches Entladen / Laden der Kernelmodule beim Bootvorgang (Achtung: funktioniert noch nicht!)

```
$ sudo nano /etc/udev/rules.d/20-i2c-usb.rules

#CH341 I2C to USB adapter
ACTION=="add", SUBSYSTEM=="i2c-dev", ATTRS{idVendor}=="1a86", ATTRS{idProduct}=="5512", RUN+="/sbin/modprobe i2c-ch341-usb"
```

Neustart des ```udev```-Daemons:

```
$ sudo systemctl restart udev.service
```



In [1]:
import smbus2
import bme280
import time

port = 11
address = 0x76
bus = smbus2.SMBus(port)

INTERVAL = 1.0

calibration_params = bme280.load_calibration_params(bus, address)

#while True:
# the sample method will take a single reading and return
# a compensated_reading object
data = bme280.sample(bus, address, calibration_params)

# the compensated_reading class has the following attributes
#print(data.id)
#print(data.timestamp)
#print(data.temperature)
#print(data.pressure)
#print(data.humidity)

print("{time:s} Temperature: {temperature:.2f} °C, Pressure: {pressure:.2f} hPa, Humidity: {humidity:.2f} % rH".format(time=data.timestamp.strftime('%Y-%m-%d %H:%M:%S'), temperature=data.temperature, pressure=data.pressure, humidity=data.humidity))
    
#    time.sleep(INTERVAL)

# there is a handy string representation too
#print(data)

2021-05-20 18:27:04 Temperature: 22.58 °C, Pressure: 1004.87 hPa, Humidity: 39.59 % rH


In [4]:
print(data)

compensated_reading(id=7a02a4d9-733b-413d-9605-64995e65a33f, timestamp=2021-05-20 15:26:17.315919, temp=20.535 °C, pressure=1005.39 hPa, humidity=38.30 % rH)


In [5]:
data.timestamp.strftime('%Y-%m-%d %H:%M:%S')

'2021-05-20 15:26:17'

In [14]:
import curses

In [15]:
time.time()

1562942569.8146622

In [18]:
data.timestamp.timestamp()

1562928535.463544